In [6]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def myStrategy(daily, minutely, open, lr, sr):
    arr = []
    short = []
    for i in range(lr,daily.shape[0]):
        avg = np.mean(daily['open'][i-lr:i].values)
        arr.append(avg)
    for i in range(sr,daily.shape[0]):
        avg = np.mean(daily['open'][i-sr:i].values)
        short.append(avg)
    return arr, short

def data_gen(daily, start):
    train_x = []
    train_y = []
    for i in range(start, daily.shape[0]-1):
        train_x.append(daily[i-start:i].reshape(start*4))
        train_y.append(daily[i+1][3])
    return np.array(train_x), np.array(train_y)

In [7]:
daily = pd.read_csv("TX_daily.csv")
minutely = pd.read_csv("TX_minutely.csv")
minute_np = daily.to_numpy() #daily or minutely
minute_np = np.delete(minute_np,0,axis=1)
minute_np = np.delete(minute_np,4,axis=1)
train_x, train_y = data_gen(minute_np,20)

In [8]:
count = 0
nan_arr = []
for i in range(train_x.shape[0]):
    for j in range(80):
        if np.isnan(train_x[i][j]):
            nan_arr.append(i)
            break
nan_arr = np.array(nan_arr)
train_x = np.delete(train_x, nan_arr, axis = 0)
train_y = np.delete(train_y, nan_arr, axis = 0)

C:\Users\ultim\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  if __name__ == '__main__':
C:\Users\ultim\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
  # Remove the CWD from sys.path while we load stuff.


In [9]:
train_x.shape

(2181, 80)

In [10]:
weight = np.zeros(train_x.shape[1])
for i in range(len(weight)):
    weight[i] = 1

lr = 1e-14
for i in range(10000):
    if i % 1000 == 0:
        print(i)
    y = np.dot(train_x, weight)
    loss = y - train_y
    gradient = 2*np.dot(train_x.T, loss)
    #print(gradient)
    weight = weight - lr*gradient

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [11]:
print(weight) # low的權重很重，感覺其他方式要用它?!

[0.007291727294631128 0.002729207030992755 0.012390487875319602
 0.007351601337959795 0.0071709457432260456 0.0026063372729191146
 0.012252552207491012 0.007252655634525044 0.007081466834350503
 0.002509392788626616 0.012128532038257757 0.00715199953917807
 0.007024223566102112 0.002485717919912585 0.012129498372252618
 0.007162853239488879 0.007071042404392404 0.0024833292189513213
 0.012187282335265849 0.00716862353695612 0.0071281036760300585
 0.0025812048977864694 0.012320667360474422 0.007378205675704321
 0.007368570835908435 0.0028256814291799334 0.012682408524187049
 0.007710517653350508 0.007806222708269248 0.003315646617366563
 0.013268097526213139 0.008260509579476874 0.008481834519974389
 0.003966508571155869 0.014002880457036869 0.008959802710885263
 0.009279078860949995 0.004690422128381189 0.014870256519270968
 0.009746106699615325 0.010166645442141808 0.00557176519009417
 0.01588564688046024 0.010748280661069426 0.01119371738527769
 0.006629006266228699 0.017003979107756

In [12]:
test_x = []
for i in range(len(daily)-21,len(daily)-1):
    test_x.append(daily.values[i][1:5])
test_x = np.array(test_x)
test_x = test_x.reshape(80)
print(np.dot(test_x.T,weight))
print(daily.values[len(daily)-1][4])

11669.495717585602
11961.0


In [15]:
np.save("weight.npy",weight)

In [17]:
for i in weight:
    print(i,end = ",")

0.007291727294631128,0.002729207030992755,0.012390487875319602,0.007351601337959795,0.0071709457432260456,0.0026063372729191146,0.012252552207491012,0.007252655634525044,0.007081466834350503,0.002509392788626616,0.012128532038257757,0.00715199953917807,0.007024223566102112,0.002485717919912585,0.012129498372252618,0.007162853239488879,0.007071042404392404,0.0024833292189513213,0.012187282335265849,0.00716862353695612,0.0071281036760300585,0.0025812048977864694,0.012320667360474422,0.007378205675704321,0.007368570835908435,0.0028256814291799334,0.012682408524187049,0.007710517653350508,0.007806222708269248,0.003315646617366563,0.013268097526213139,0.008260509579476874,0.008481834519974389,0.003966508571155869,0.014002880457036869,0.008959802710885263,0.009279078860949995,0.004690422128381189,0.014870256519270968,0.009746106699615325,0.010166645442141808,0.00557176519009417,0.01588564688046024,0.010748280661069426,0.01119371738527769,0.006629006266228699,0.017003979107756712,0.0119018774

In [47]:
lr = 365
sr = 20
ly, sy = myStrategy(daily,minutely, 0, lr, sr)
lx = range(lr,daily.shape[0])
sx = range(sr,daily.shape[0])
plt.plot(lx, ly)
plt.plot(sx, sy)
plt.show()

[9000.0 9030.0 8995.0 9020.0]
